# Part 1 Data Description

In [7]:
# Set up
library(tidyverse)
library(repr)
library(tidymodels)
setwd("/home/jovyan/work/dsci-100-2025w1-group-27")

In [11]:
# Load Data
players <- read_csv("players.csv")
sessions <- read_csv("sessions.csv")

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


## 1.1 Players Summary

In [23]:
print(players)

# A tibble: 196 × 7
   experience subscribe hashedEmail              played_hours name  gender   Age
   <chr>      <lgl>     <chr>                           <dbl> <chr> <chr>  <dbl>
 1 Pro        TRUE      f6daba428a5e19a3d475748…         30.3 Morg… Male       9
 2 Veteran    TRUE      f3c813577c458ba0dfef809…          3.8 Chri… Male      17
 3 Veteran    FALSE     b674dd7ee0d24096d1c0196…          0   Blake Male      17
 4 Amateur    TRUE      23fe711e0e3b77f1da7aa22…          0.7 Flora Female    21
 5 Regular    TRUE      7dc01f10bf20671ecfccdac…          0.1 Kylie Male      21
 6 Amateur    TRUE      f58aad5996a435f16b0284a…          0   Adri… Female    17
 7 Regular    TRUE      8e594b8953193b26f498db9…          0   Luna  Female    19
 8 Amateur    FALSE     1d2371d8a35c8831034b25b…          0   Emer… Male      21
 9 Amateur    TRUE      8b71f4d66a38389b7528bb3…          0.1 Nata… Male      47
10 Veteran    TRUE      bbe2d83de678f519c4b3daa…          0   Nyla  Female    22
# ℹ 186 

Above is a preview of the players dataset. There are a total  of 196 rows, each representing a player. Out of all columns, all columns except played_hours and Age are categorical. We can learn from finding unique categories for these columns below.

In [30]:
categoricals_unique <- players |>
select(-played_hours, -Age) |>
map(unique)
print(categoricals_unique)

$experience
[1] "Pro"      "Veteran"  "Amateur"  "Regular"  "Beginner"

$subscribe
[1]  TRUE FALSE

$hashedEmail
  [1] "f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d"
  [2] "f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9"
  [3] "b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28"
  [4] "23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5"
  [5] "7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e"
  [6] "f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977"
  [7] "8e594b8953193b26f498db95a508b03c6fe1c24bb5251d392c18a0da9a722807"
  [8] "1d2371d8a35c8831034b25bda8764539ab7db0f63938696917c447128a2540dd"
  [9] "8b71f4d66a38389b7528bb38ba6eb71157733df7d1740371852a797ae97d82d1"
 [10] "bbe2d83de678f519c4b3daa7265e683b4fe2d814077f9094afd11d8f217039ec"
 [11] "7a4686586d290c67179275c7c3dfb4ea02f4d317d9ee0e2cee98baa27877a875"
 [12] "4caa42e1b20511552434978171dcf7283fb6eb857eb871327e39bfc328002309"
 [13] "4bed

Running the code above shows that experience has five categories, subscribed column is either true or false, and gender has seven categories. Hashed emails are unique for each player, as there are 196 hashed emails. Interestingly, none of the players have the same name either, as there are 196 different names listed.

## 1.2 Sessions Summary

In [31]:
print(sessions)

# A tibble: 1,535 × 5
   hashedEmail         start_time end_time original_start_time original_end_time
   <chr>               <chr>      <chr>                  <dbl>             <dbl>
 1 bfce39c89d6549f2bb… 30/06/202… 30/06/2…       1719770000000     1719770000000
 2 36d9cbb4c6bc0c1a69… 17/06/202… 17/06/2…       1718670000000     1718670000000
 3 f8f5477f5a2e53616a… 25/07/202… 25/07/2…       1721930000000     1721930000000
 4 bfce39c89d6549f2bb… 25/07/202… 25/07/2…       1721880000000     1721880000000
 5 36d9cbb4c6bc0c1a69… 25/05/202… 25/05/2…       1716650000000     1716650000000
 6 bfce39c89d6549f2bb… 23/06/202… 23/06/2…       1719160000000     1719160000000
 7 fd6563a4e0f6f42735… 15/04/202… 15/04/2…       1713170000000     1713170000000
 8 ad6390295640af1ed0… 21/09/202… 21/09/2…       1726880000000     1726890000000
 9 96e190b0bf3923cd8d… 21/06/202… 21/06/2…       1718940000000     1718940000000
10 36d9cbb4c6bc0c1a69… 16/05/202… 16/05/2…       1715840000000     1715840000000
# ℹ 1,

Above is a preview of the sessions dataset. There are a total of 1535 rows, each representing a play session. start_time and end_time are human readable forms of orginal_start_time and original_end_time. By finding the min/max of the original start/ends, respectively, then reading the human-readables of those rows, we can find the time range in which the data was collected.

In [61]:
# ?which
min_start<- which(sessions$original_start_time == min(sessions$original_start_time))
max_end <- which(sessions$original_end_time == max(sessions$original_end_time, na.rm = TRUE))

slice(sessions, min_start) |>
slice(1) |>
select(start_time)

slice(sessions, max_end) |>
select(end_time)

start_time
<chr>
06/04/2024 09:35


end_time
<chr>
26/09/2024 07:39


As shown, the earliest datapoint started around June 4th, and the latest datapoint ended round September 26th.

# Part 2 Questions

## 2.1 Broad Question

We would like to know which "kinds" of players are most likely to contribute a large amount of data so that we can target those players in our recruiting efforts.

## 2.2 Specific Question

We would like to know the the relationship between age and playtime.

## 2.3 What data to use

We have data for the age of each player, as well as their play time. This will allow us to plot a scatterplot,and possibly also use a regression model if we extend beyond just the sepcific question.